# 🐫 CAMEL Task Generation Cookbook

You can also check this cookbook in colab [here](https://colab.research.google.com/drive/1K5I7hXUQusyRw-Q5kd_NiP2fJYHew65M?usp=sharing)

## In this tutorial, we will focus on demonstrating how to use the task module in the CAMEL framework. We will guide you through creating, evolving, and decomposing tasks to illustrate how the task module can be utilized for efficient task management in agent-based systems.

### Sections included:

- Setting up a ChatAgent with the CAMEL framework
- Creating a Task and evolving it with the agent using TaskManager
- Task decomposition using the CAMEL task module

Let's go step by step!

## Step 1: Import necessary CAMEL modules

First, we need to import the required CAMEL modules for creating the ChatAgent and handling tasks.

In [2]:
!pip install camel-ai==0.1.6.8

  Using cached ipykernel-6.29.5-py3-none-any.whl.metadata (6.3 kB)
Using cached ipykernel-6.29.5-py3-none-any.whl (117 kB)
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.5.6
    Uninstalling ipykernel-5.5.6:
      Successfully uninstalled ipykernel-5.5.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel==5.5.6, but you have ipykernel 6.29.5 which is incompatible.


In [3]:
from camel.agents import ChatAgent
from camel.configs import ChatGPTConfig
from camel.messages import BaseMessage
from camel.models import ModelFactory
from camel.tasks import (
    Task,
    TaskManager,
)
from camel.types import (
    ModelPlatformType,
    ModelType,
)

Set your OpenAI key

In [4]:
import os
from getpass import getpass

# Prompt for the API key securely
openai_api_key = getpass('Enter your API key: ')
os.environ["OPENAI_API_KEY"] = openai_api_key

Enter your API key: ··········


## Step 2: Set up the Large Language Model (LLM)

Next, we set up the model configuration. We are using a CAMEL LLM (GPT-4O Mini in this case) for our assistant agent. The configuration is designed to ensure the agent’s behavior remains deterministic by setting temperature=0.0, meaning no randomness will be introduced in the responses.

In [5]:
# Model configuration for the assistant
assistant_model_config = ChatGPTConfig(
    temperature=0.0,  # Set to 0 for deterministic output
)

# Create the model using the configuration
model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_4O_MINI,
    model_config_dict=assistant_model_config.as_dict(),
)

## Step 3: Initialize the ChatAgent

We now create a ChatAgent using the previously defined model. This agent will interact with tasks in the CAMEL framework, following the role of a personal math tutor and programmer.

In [6]:
# Set up the assistant's system message
assistant_sys_msg = BaseMessage.make_user_message(
    role_name="Teacher",
    content=("You are a personal math tutor and programmer."),
)

# Initialize the ChatAgent
agent = ChatAgent(assistant_sys_msg, model)
agent.reset()  # Reset the agent's internal state

## Step 4: Evolve and decompose tasks

We now create a Task that represents a math problem for the assistant to solve. In this case, we are asking the assistant to calculate how much Weng earned for babysitting based on her hourly rate and the time she worked.

In [7]:
# Create a Task for the agent to solve
task = Task(
    content="Weng earns $12 an hour for babysitting. Yesterday, she just did 51 minutes of babysitting. How much did she earn?",
    id="0",  # Task identifier
)

# Print the task to see the original form
print(task.to_string())

Task 0: Weng earns $12 an hour for babysitting. Yesterday, she just did 51 minutes of babysitting. How much did she earn?



### Evolve the Task

We can evolve the task using the TaskManager, which allows the agent to potentially update or reframe the task based on its internal logic and context.

In [12]:
task_manager = TaskManager(task)

evolved_task = task_manager.evolve(task, agent=agent)
if evolved_task is not None:
    print(evolved_task.to_string())
else:
    print("Evolved task is None.")

Task 0.0: Weng earns $12 an hour for babysitting. This month, she has a special promotion where she earns an additional $2 per hour for every hour worked over 10 hours. Last week, she babysat for 3 hours and 30 minutes on Monday, 2 hours and 15 minutes on Wednesday, and 4 hours on Saturday. How much did she earn for the week, including any bonuses from the promotion?



### Decompose the Task

Sometimes, tasks are complex and need to be broken down into smaller subtasks. We use decompose() to allow the agent to split the original task into simpler parts.

In [13]:
new_tasks = task.decompose(agent=agent)
for t in new_tasks:
    print(t.to_string())

Task 0.0: Convert the babysitting time from minutes to hours.

Task 0.1: Calculate the earnings based on the hourly rate of $12 per hour.

Task 0.2: Multiply the number of hours worked by the hourly rate to find the total earnings.

